# TF-IDF and BM25

Using the classic Cranfield dataset, this notebook shows how to use [TF-IDF](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) and [BM25](https://pypi.org/project/rank-bm25) to calculate the similarity scores between a query and the documents and show the evaluation scores, i.e., precision and recall. Note that the ranking of the returned documents is not yet considered.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# load data into dataframes
docs = pd.read_json("data/cranfield_docs.json")
queries = pd.read_json("data/cranfield_queries.json")
relevance = pd.read_json("data/cranfield_relevance.json")

In [3]:
docs.head()

,id,author,bibliography,body,title
0,1,"brenckman,m.","j. ae. scs. 25, 1958, 324.",experimental investigation of the aerodynamics...,experimental investigation of the aerodynamics...
1,2,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...,simple shear flow past a flat plate in an inco...
2,3,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...,the boundary layer in simple shear flow past a...
3,4,"yen,k.t.","j. ae. scs. 22, 1955, 728.",approximate solutions of the incompressible la...,approximate solutions of the incompressible la...
4,5,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...,one-dimensional transient heat conduction into...


In [4]:
queries.head()

,query_id,query
0,1,what similarity laws must be obeyed when const...
1,2,what are the structural and aeroelastic proble...
2,3,what problems of heat conduction in composite ...
3,4,can a criterion be developed to show empirical...
4,5,what chemical kinetic system is applicable to ...


In [5]:
queries.set_index('query_id', inplace=True)
queries

,query
query_id,
1,what similarity laws must be obeyed when const...
2,what are the structural and aeroelastic proble...
3,what problems of heat conduction in composite ...
4,can a criterion be developed to show empirical...
5,what chemical kinetic system is applicable to ...
...,...
221,papers applicable to this problem (calculation...
222,has anyone investigated the shear buckling of ...
223,papers on shear buckling of unstiffened rectan...


In [6]:
queries['query'][1]

'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .'

In [7]:
relevance.head()

,query_id,r_score,doc_id
0,1,2,184
1,1,2,29
2,1,2,31
3,1,3,12
4,1,3,51


In [8]:
train_docs = docs['body'].tolist()

In [9]:
# only need nltk if custom tokenizer is used
import nltk
nltk.download('popular')  # download the nltk packages locally

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/harrywang/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/harrywang/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/harrywang/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/harrywang/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/harrywang/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/harrywang/nltk_data...
[nltk_data]    |   Package movie_

True

In [10]:
# https://tartarus.org/martin/PorterStemmer/def.txt
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
import re # regular expression


def stemming_tokenizer(str_input):
    words = re.sub(r"[^A-Za-z\-]", " ", str_input).lower().split() # delete non letter charactors
    #words = re.sub(r"[^A-Za-z0-9\-]", " ", str_input).lower().split() # include numbers
    words = [porter_stemmer.stem(word) for word in words]
    return words

In [11]:
# train the tf-idf model
# max_df: 0.9 - discard the term if the term appears more than 80% of all documents
# min_idf: 0.1 - discard the term if the teram appears in less than 20% of all documents 
# ngram_range: (1,3) - try unigrams, bigrams and trigrams
# tokenizer=stemming_tokenizer - use customized tokenizer
# use_idf=True, norm='l2' default
# using stemming_tokenizer improves precision/recall a lot!

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity 

tfidf_vectorizer = TfidfVectorizer(
        tokenizer=stemming_tokenizer, 
        max_df=0.9, 
        min_df=0.1, 
        stop_words='english',
        use_idf=True, 
        ngram_range=(1, 3)
)


tfidf_vect= tfidf_vectorizer.fit(train_docs)

In [12]:
# number of words/features
len(tfidf_vect.get_feature_names())

130

In [13]:
# testing data
# first sentence is the query
# rest are the documents
test_texts = [
    "photo-thermoelastic investigation",
    "a simple model study of transient temperature and thermal stress distribution due to aerodynamic heating .   the present work is concerned with the determination of transient temperatures and thermal stresses in simple models intended to simulate parts or the whole of an aircraft structure of the built- up variety subjected to aerodynamic heating .   the first case considered is that of convective heat transfer into one side of a flat plate, representing a thick skin, and the effect of the resulting temperature distribution in inducing thermal stresses associated with bending restraint at the plate edges . numerical results are presented for the transient temperature differentials in the plate when the environment temperature first increases linearly with time and then remains constant, the period of linear increase representing the time of acceleration of the aircraft .  corresponding thermal stress information is presented .   the second case is that of the wide-flanged i-beam with convective heat transfer into the outer faces of the flanges .  numerical results are presented for transient temperature differentials for a wide range of values of the applicable parameters and for an environment temperature variation as described above . corresponding thermal stresses in a beam of infinite length are determined .  a theoretical analysis of the stress distribution in a beam of finite length is carried out and numerical results obtained for one case .",
    "photo-thermoelastic investigation of transient thermal stresses in a multiweb wing structure .   photothermoelastic experiments were performed on a long multiweb wing model for which a theoretical analysis is available in the literature .  the experimental procedures utilized to simulate the conditions prescribed in the theory are fully described .   correlation of theory and experiment in terms of dimensionless temperature, stress, time, and biot number revealed that the theory predicted values higher than the experimentally observed maximum thermal stresses at the center of the web .  ",
]

test_result = tfidf_vect.transform(test_texts)
pd.DataFrame(test_result.toarray(), columns=tfidf_vect.get_feature_names())

,aerodynam,agreement,air,analysi,angl,appli,applic,approxim,assum,base,...,type,use,valu,vari,variou,veloc,wa,wall,wave,wing
0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.155785,0.0,0.0,0.066279,0.0,0.0,0.072221,0.0,0.0,0.0,...,0.0,0.0,0.063679,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.0,0.0,0.147704,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.141909,0.0,0.0,0.0,0.0,0.0,0.0,0.322923


In [14]:
# cosine similarity for the testing data
test_similarity = cosine_similarity(test_result)
pd.DataFrame(test_similarity)

,0,1,2
0,1.000000,0.00000,0.132622
1,0.000000,1.00000,0.413150
2,0.132622,0.41315,1.000000


In [15]:
# given a query and similarity_thershold return the ids of docs
# this is the relevant docs that based on our tf-idf algorithm for the query
def get_doc_ids_tfidf(query_id, similarity_threshold):
    
    all_docs = train_docs.copy() # make a copy of all docs list
    all_docs.insert(0, queries['query'][query_id]) # inser the current query at the begining of the list
    
    test_result = tfidf_vect.transform(all_docs) # generate the tfidf matrix
    # pd.DataFrame(test_result.toarray(), columns=tfidf_vect.get_feature_names())
    df = pd.DataFrame(cosine_similarity(test_result)) # calculate the pair-wise similarity and convert to df
    df = df.rename(columns = {0:'similarity'}) # rename the first column 
    df = df.sort_values('similarity', ascending=False) # sort the result based on similarity score
    df_filtered = df[df['similarity']>similarity_threshold] # filter the result based on similarity threshold
    returned_doc_ids_list = df_filtered.index.tolist() # get the ids of the returned docs
    return returned_doc_ids_list

In [16]:
# get the doc ids with relevance score 
# this is the ground truth of relevance docs for the query based on the human annotated data
def get_true_doc_ids(query_id, relevance_threshold):
    # filter based on r_score (1, 2, 3, or 4) and relevance_threshold
    true_doc_ids = relevance[(relevance['query_id']==query_id) & (relevance['r_score']<=relevance_threshold)]
    true_doc_ids_list = true_doc_ids['doc_id'].to_list()
    return true_doc_ids_list

In [17]:
# calculate evaluation scores: precision and recall

def show_eval_scores(returned_doc_ids_list, true_doc_ids_list):
    
    # true positive 
    tp = [x for x in true_doc_ids_list if x in returned_doc_ids_list]
    #tp.sort()
    #print(tp, len(tp))
    
    # false positive
    fp = [x for x in returned_doc_ids_list if x not in tp]
    #fp.sort()
    #print(fp, len(fp))
    
    # false negative
    fn = [x for x in true_doc_ids_list if x not in tp]
    #fn.sort()
    #print(fn, len(fn))
    
    precision = len(tp)/(len(tp)+len(fp))
    recall = len(tp)/(len(tp)+len(fn))


    print(f'precision is {precision:.3%}')
    print(f'recall is {recall:.3%}')
    
    return precision, recall

In [18]:
# utility function to put everything together
def show_result_tfidf(query_id, similarity_threshold, relevance_threshold):
    
    print(f"query: {queries['query'][query_id]}")
    print('calculating results......')
    # we set a similarity threshold and get the ids of those documents
    # similarity_threshold 0.65 is used in https://www.aaai.org/Papers/FLAIRS/2003/Flairs03-082.pdf
    returned_doc_ids_list = get_doc_ids_tfidf(query_id, similarity_threshold)
    
    print(f'total # of returned docs: {len(returned_doc_ids_list)}')

    # we choose relevance_threshold = 3, relevance 1, 2 and 3 mean relevant for returned documents
    # see readme for definitions about r_score
    true_doc_ids_list = get_true_doc_ids(query_id, relevance_threshold)
    print(f'total # of true docs: {len(true_doc_ids_list)}')

    print(f'similarity_threshold is {similarity_threshold} and relevance_threshold is {relevance_threshold}')

    show_eval_scores(returned_doc_ids_list, true_doc_ids_list)

In [19]:
# given a query and similarity_thershold return the ids of docs
# this is the relevant docs that based on our tf-idf algorithm for the query

from rank_bm25 import BM25Okapi

def get_doc_ids_bm25(query_id, bm25_top_n):
    
    all_docs = train_docs.copy() # make a copy of all docs list
    tokenized_corpus = [ stemming_tokenizer(doc) for doc in all_docs]
    bm25 = BM25Okapi(tokenized_corpus)
    
    query = queries['query'][query_id]
    tokenized_query = stemming_tokenizer(query)
    
    doc_scores = bm25.get_scores(tokenized_query)
    df = pd.DataFrame(doc_scores)
    df = df.rename(columns = {0:'bm25_score'}) # rename the first column 
    df = df.sort_values('bm25_score', ascending=False) # sort the result based on similarity score
    df_filtered = df[0:bm25_top_n] # filter the result based on similarity threshold
    returned_doc_ids_list = df_filtered.index.tolist() # get the ids of the returned docs
    return returned_doc_ids_list

In [20]:
# utility function to put everything together
def show_result_bm25(query_id, bm25_top_n, relevance_threshold):
    
    print(f"query: {queries['query'][query_id]}")
    print('calculating results......')
    # we set a bm25_top_n and get the ids of those documents
    returned_doc_ids_list = get_doc_ids_bm25(query_id, bm25_top_n)
    print(f'total # of returned docs: {len(returned_doc_ids_list)}')
    
    # we choose relevance_threshold = 3, relevance 1, 2 and 3 mean relevant for returned documents
    # see readme for definitions about r_score
    true_doc_ids_list = get_true_doc_ids(query_id, relevance_threshold)
    print(f'total # of true docs: {len(true_doc_ids_list)}')
    
    print(f'bm25 top_n is {bm25_top_n} and relevance_threshold is {relevance_threshold}')

    show_eval_scores(returned_doc_ids_list, true_doc_ids_list)

In [21]:
# query_id = 1, similarity_threshold = 0.3, relevance_threshold = 3
show_result_tfidf(1, 0.3, 3)
show_result_bm25(1, 30, 3)

query: what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .
calculating results......
total # of returned docs: 30
total # of true docs: 22
similarity_threshold is 0.3 and relevance_threshold is 3
precision is 23.333%
recall is 31.818%
query: what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .
calculating results......
total # of returned docs: 30
total # of true docs: 22
bm25 top_n is 30 and relevance_threshold is 3
precision is 6.667%
recall is 9.091%


In [22]:
# query_id = 2, similarity_threshold = 0.3, relevance_threshold = 3
show_result_tfidf(2, 0.3, 3)
show_result_bm25(2, 36, 3)

query: what are the structural and aeroelastic problems associated with flight of high speed aircraft .
calculating results......
total # of returned docs: 36
total # of true docs: 21
similarity_threshold is 0.3 and relevance_threshold is 3
precision is 11.111%
recall is 19.048%
query: what are the structural and aeroelastic problems associated with flight of high speed aircraft .
calculating results......
total # of returned docs: 36
total # of true docs: 21
bm25 top_n is 36 and relevance_threshold is 3
precision is 2.778%
recall is 4.762%
